In [15]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import re

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
  print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
  print("OpenAI API Key not set")

if anthropic_api_key:
  print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
  print("Anthropic API Key not set (and this is optional)")

MODEL_GPT = 'gpt-5-mini'
MODEL_CLAUDE = 'claude-sonnet-4-6'

anthropic_url = "https://api.anthropic.com/v1/"

openai = OpenAI()
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

MODELS = {
  "claude-sonnet-4-6": anthropic,
  "gpt-5-mini": openai
}

system_prompt = """
  You are a helpful assistant who excels in generating code snippets, writing unit tests, and providing explanations for code. You can also help with debugging and optimizing code. Always provide clear and concise responses, and include code examples when relevant.
"""

def generate_docstring(code_snippet, model_name):
  model = MODELS.get(model_name)
  if not model:
    return "Model not found."

  docstring_prompt = f"""
    Please provide docstrings for the following code snippet. The docstring should be clear and concise, explaining the purpose of the code, its parameters, return values, and any exceptions it may raise. Use the appropriate format for the programming language of the code snippet. Your response should include the output code language in Gradio code format followed by a single whitespace, then the input code snippet, and should not include any additional explanations or comments.

    {code_snippet}
  """

  response = model.chat.completions.create(
    model=model_name,
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": docstring_prompt}],
    max_tokens=500
  )
  return response.choices[0].message.content.strip()

def generate_unit_tests(code_snippet, model_name):
  model = MODELS.get(model_name)
  if not model:
    return "Model not found."

  unit_test_prompt = f"""
    Please write unit tests for the following code snippet. The tests should cover various cases, including edge cases, and should be written in a clear and concise manner. Use the appropriate testing framework for the programming language of the code snippet. Your response should include the output code language in Gradio code format followed by a single whitespace, then the unit tests, and should not include any additional explanations or comments.

    {code_snippet}
  """

  response = model.chat.completions.create(
    model=model_name,
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": unit_test_prompt}]
  )
  return response.choices[0].message.content.strip()

def extract_code_and_language(output):
  match = re.search(r"```(\w+)?\n(.*?)```", output, re.DOTALL)
  if match:
    language = match.group(1) or "python"
    code = match.group(2).strip()
    return code, language
  return output, "python"

def display_code_output(output):
  code, language = extract_code_and_language(output)
  return gr.update(value=code, language=language)

with gr.Blocks() as ui:
  gr.Markdown("## Code Helper Assistant")
  with gr.Row():
    model_selector = gr.Radio(label="Select Model", choices=list(MODELS.keys()), value=MODEL_CLAUDE)
  with gr.Row(equal_height=True):
    with gr.Column():
      code_input = gr.TextArea(label="Enter your code snippet here", lines=10)
    with gr.Column():
      output = gr.Code(label="Output", lines=20)
  with gr.Row():
    generate_docstring_btn = gr.Button("Generate Docstrings")
    generate_unit_tests_btn = gr.Button("Generate Unit Tests")

  generate_docstring_btn.click(
    lambda code, model: display_code_output(generate_docstring(code, model)),
    inputs=[code_input, model_selector],
    outputs=output
  )
  generate_unit_tests_btn.click(
    lambda code, model: display_code_output(generate_unit_tests(code, model)),
    inputs=[code_input, model_selector],
    outputs=output
  )

ui.launch(inbrowser=True)

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
